<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
# Daniel Alejandro Cárdenas Niño

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [2]:
import pandas as pd
import tweepy

In [3]:
# Leer las llaves
keys = pd.read_csv('C:/Users/dcard/OneDrive/Escritorio/NPL/dani.csv', header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [4]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
tweets = tweepy.Cursor(api.search,
                       q=['Copa america colombia -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,10000km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [6]:
# Objeto generador
t = [tweet for tweet in tweets]

In [7]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweets,date,city,user_location,id,link
0,Si con esto no quitan la copa América de Colom...,2021-05-20 00:10:53,Bogotá,"Cartagena, Colombia",1395170125939191812,https://twitter.com/i/web/stuatus/139517012593...
1,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,2021-05-20 00:10:00,Bogotá,Colombia,1395169901661368323,https://twitter.com/i/web/stuatus/139516990166...
2,Que la @CONMEBOL no realice la Copa América en...,2021-05-20 00:02:01,Bogotá,"Bogota ,Colombia",1395167894833672195,https://twitter.com/i/web/stuatus/139516789483...
3,#NoALaCopaAmericaEnColombia \nQue la @CONMEBOL...,2021-05-19 23:55:17,Bogotá,Holanda,1395166200536182790,https://twitter.com/i/web/stuatus/139516620053...
4,Que la @CONMEBOL no realice la Copa América en...,2021-05-19 23:42:06,Bogotá,Cali,1395162881600151557,https://twitter.com/i/web/stuatus/139516288160...


In [8]:
import snscrape.modules.twitter as sntwitter

In [9]:
maxTweets = 100
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#NoCopaAmericaEnColombia ' + 'since:2021-04-01 until:2021-05-18').get_items()) :
        if i > maxTweets :
            break
        print(tweet)
        print(tweet.date)
        print(tweet.content)
        print("*"*100)
        print()
        
        
data = data[['tweets', 'date']]
data.head()

https://twitter.com/yaal33/status/1394127550088110084
2021-05-17 03:08:04+00:00
#nocopaamericaencolombia #SOSColombiaDDDHH https://t.co/qRiwQhjM1k
****************************************************************************************************

https://twitter.com/YunnanMusik/status/1394068679101403139
2021-05-16 23:14:08+00:00
Esperamos q nos puedan apoyar jugadores, directores e integrantes de cada selección a la voz del pueblo colombiano, ya que en Colombia estamos en un periodo de recambio y necesitamos espacio para ello, por eso NO a la copa en Colombia.
#SOSColombiaDDHH  
#NoCopaAmericaEnColombia
****************************************************************************************************

https://twitter.com/DiFirulais_Col/status/1394055585692360708
2021-05-16 22:22:06+00:00
El ministro de deporte “insiste en su #CopaAmerica2021”
Estos hijueputas fascistas se ven tan tiernos, citando palabras del ex-guerrillero Nelson Mandela.🤣🤣🤣
#NoCopaAmericaEnColombia
#NoCopaAmeric

https://twitter.com/CamiloDiazQ/status/1392903127901290507
2021-05-13 18:02:39+00:00
@CONMEBOL --&gt; Vergüenza!!!! Entienden eso??? #NoCopaAmericaEnColombia https://t.co/kyliqd5Dkz
****************************************************************************************************



,tweets,date
0,Si con esto no quitan la copa América de Colom...,2021-05-20 00:10:53
1,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,2021-05-20 00:10:00
2,Que la @CONMEBOL no realice la Copa América en...,2021-05-20 00:02:01
3,#NoALaCopaAmericaEnColombia \nQue la @CONMEBOL...,2021-05-19 23:55:17
4,Que la @CONMEBOL no realice la Copa América en...,2021-05-19 23:42:06


In [10]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data


,tweets,date,texto_limpio,links,hashtags,mentions
0,Si con esto no quitan la copa América de Colom...,2021-05-20 00:10:53,Si con esto no quitan la copa América de Colom...,[],[],[]
1,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,2021-05-20 00:10:00,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,[https://t.co/M43cvmMo8r],[],[@VBarCaracol]
2,Que la @CONMEBOL no realice la Copa América en...,2021-05-20 00:02:01,Que la no realice la Copa América en Colombia...,[https://t.co/6khZBP94qB],[#NoALaCopaAméricaEnColombia],"[@CONMEBOL, @ChangeorgCO]"
3,#NoALaCopaAmericaEnColombia \nQue la @CONMEBOL...,2021-05-19 23:55:17,\nQue la no realice la Copa América en Colom...,[https://t.co/9j6tHyzxAx],"[#NoALaCopaAmericaEnColombia, #NoALaCopaAméric...","[@CONMEBOL, @ChangeorgCO]"
4,Que la @CONMEBOL no realice la Copa América en...,2021-05-19 23:42:06,Que la no realice la Copa América en Colombia...,[https://t.co/7lkytS8Ulw],[#NoALaCopaAméricaEnColombia],"[@CONMEBOL, @ChangeorgCO]"
...,...,...,...,...,...,...
95,Que la @CONMEBOL no realice la Copa América en...,2021-05-19 17:15:00,Que la no realice la Copa América en Colombia...,[https://t.co/wZto5eQDyn],[#NoALaCopaAméricaEnColombia],"[@CONMEBOL, @ChangeorgCO]"
96,¡No a la copa América en #Colombia!\n\n¡Falta!...,2021-05-19 17:09:07,¡No a la copa América en \n\n¡Falta! ❌🚫☢️⚠️\nN...,[https://t.co/qutrAlTWg5],[#Colombia!],[]
97,#Ahora Las barras sociales se unen pacíficamen...,2021-05-19 17:02:56,Las barras sociales se unen pacíficamente al ...,[https://t.co/2zbPUPpxbu],"[#Ahora, #ParoNacional, #Barranqueños, #ParoNa...","[@PrensaRural, @marchapatriota, @CopaAmerica]"
98,Tocaría poner otras reglas pa la copa América ...,2021-05-19 16:57:13,Tocaría poner otras reglas pa la copa América ...,[],[],[]


In [11]:

import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,Si con esto no quitan la copa América de Colom...,2021-05-20 00:10:53,Si con esto no quitan la copa América de Colom...,[],[],[],[]
1,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,2021-05-20 00:10:00,🇨🇴⚽ El jugador del Al-Hilal de la Liga de Arab...,[https://t.co/M43cvmMo8r],[],[@VBarCaracol],"[⚽, 📻]"
2,Que la @CONMEBOL no realice la Copa América en...,2021-05-20 00:02:01,Que la no realice la Copa América en Colombia...,[https://t.co/6khZBP94qB],[#NoALaCopaAméricaEnColombia],"[@CONMEBOL, @ChangeorgCO]",[]
3,#NoALaCopaAmericaEnColombia \nQue la @CONMEBOL...,2021-05-19 23:55:17,\nQue la no realice la Copa América en Colom...,[https://t.co/9j6tHyzxAx],"[#NoALaCopaAmericaEnColombia, #NoALaCopaAméric...","[@CONMEBOL, @ChangeorgCO]",[]
4,Que la @CONMEBOL no realice la Copa América en...,2021-05-19 23:42:06,Que la no realice la Copa América en Colombia...,[https://t.co/7lkytS8Ulw],[#NoALaCopaAméricaEnColombia],"[@CONMEBOL, @ChangeorgCO]",[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [13]:
data['fecha'] = data.date.dt.date
temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')
temp

,fecha,cnt
0,2021-05-19,97
1,2021-05-20,3


In [14]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [15]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por día')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)